In [1]:
import pandas as pd
import tensorflow as tf

from sklearn.model_selection import train_test_split

In [2]:
from tensorflow.keras.models import load_model
model=load_model('lstm.h5')

In [3]:
test_ds='../data/test_data.csv'
test_df=pd.read_csv(test_ds)[['content']]
test_df

,content
0,银华旗下两基金开通转换 银华基金日前发布公告，自3月12日起开通银华领先策略、银华增强收益与...
1,深圳楼市现拐点价涨量跌 购房者对房价敏感 据中央电视台“朝闻天下”栏目消息，深圳楼市在今年上...
2,今年以来封基净值平均增长12.46% 本报讯 与开放式基金一样，今年以来封基的表现也很抢眼。...
3,现场：斯科拉扭伤离场检查 洛瑞迟到三分杯水车薪新浪体育讯北京时间2月28日消息，火箭背靠背客...
4,120G硬盘高清DV 索尼XR500E单机8650元 作者：李金昊 [北京行情]...
...,...
19995,张世礼：社会要尊重设计师劳动 不搞恶性竞争新浪家居杨轶讯 2010年1月22日，新浪乐居•里...
19996,搞定Icon的万能尖货Gwen Stefani “娘”了 没救了Gwen Stefani做了...
19997,专家称开发商炮制故事助推去年房价本报北京1月6日电(记者李松涛)在某论坛上，国家信息中心专家...
19998,吴尊韩庚培养默契 被高晓松关在房里对视聊天成都商报(微博)讯(记者 张世豪 实习生 李潇 摄...


In [4]:
## 大于4k 长度的比例
#print(len(test_df[test_df['content'].str.len()>4000]['content'])/len(test_df))
## 截断到 4k
idxs=test_df.loc[test_df['content'].str.len()>4000].index
#print(test_df.loc[test_df['content'].str.len()>4000].index)
test_df.loc[test_df['content'].str.len()>4000,'content']=test_df.loc[test_df['content'].str.len()>4000,'content'].apply(lambda x:x[:4000].rsplit('。',1)[0])
print(len(test_df[test_df['content'].str.len()>4000]['content'])/len(test_df))
test_df.sample(10)
test_df.shape

0.0


(20000, 1)

In [5]:
import re
#训练集文本预处理
## 文本处理
### 函数：删除除字母,数字，汉字以外的所有符号
def remove_punctuation(line):
    line = str(line)
    if line.strip()=='':
        return ''
    rule = re.compile(u"[^a-zA-Z0-9\u4E00-\u9FA5]")
    line = rule.sub('',line)
    return line

### 函数：获取停用词
def get_stopwords_list(filepath):  
    stopwords = [line.strip() for line in open(filepath, 'r', encoding='utf-8').readlines()]  
    return stopwords 

In [6]:
#删除除字母,数字，汉字以外的所有符号删除除字母,数字，汉字以外的所有符号
test_df['cleaned_content']=test_df['content'].apply(remove_punctuation)
test_df.sample(10)

,content,cleaned_content
17646,业余爱好做家具 大学老师当木匠自制家具抢手刘松果本来是重庆一所大学的美术老师，业余爱好做家具...,业余爱好做家具大学老师当木匠自制家具抢手刘松果本来是重庆一所大学的美术老师业余爱好做家具不过...
14213,组图：学日本女模穿内衣迷死男人导语：男人总是对女优趋之若鹜，如果你总是担心自己的抱歉身材吸引...,组图学日本女模穿内衣迷死男人导语男人总是对女优趋之若鹜如果你总是担心自己的抱歉身材吸引不了他...
1513,阿帅再三强调防守带来自信 最后11场不容任何闪失新浪体育讯北京时间3月23日(休斯敦当地时间...,阿帅再三强调防守带来自信最后11场不容任何闪失新浪体育讯北京时间3月23日休斯敦当地时间3月...
5512,信达澳银领先增长调整基金经理 增聘王战强全景网12月25日讯 信达澳银基金今日公告，旗下信达...,信达澳银领先增长调整基金经理增聘王战强全景网12月25日讯信达澳银基金今日公告旗下信达澳银领...
5820,宁浩《无人区》定妆照曝光 黄渤被剃光头(组图)新浪娱乐讯 由中国电影集团公司投资，宁浩执导的...,宁浩无人区定妆照曝光黄渤被剃光头组图新浪娱乐讯由中国电影集团公司投资宁浩执导的电影无人区已经...
8706,租金回本需83年 北京部分房源丧失投资价值业内人士认为宽松货币政策经济向好等因素决定房价不可...,租金回本需83年北京部分房源丧失投资价值业内人士认为宽松货币政策经济向好等因素决定房价不可能...
17961,澳高校将参考申请人诚信记录严批留学签证中新网10月18日电 据澳大利亚新快网报道，澳洲大学将...,澳高校将参考申请人诚信记录严批留学签证中新网10月18日电据澳大利亚新快网报道澳洲大学将史无...
15026,《武侠》戛纳首映反映好 王羽惠英红携手亮相新浪娱乐讯 北京时间5月15日消息，据香港东方日报...,武侠戛纳首映反映好王羽惠英红携手亮相新浪娱乐讯北京时间5月15日消息据香港东方日报报道武侠首...
4565,微博联播：于正揭秘电影《宫》 唐一菲恋八阿哥点击进入-于正微博客新浪娱乐讯 3月23日上午...,微博联播于正揭秘电影宫唐一菲恋八阿哥点击进入于正微博客新浪娱乐讯3月23日上午1057电视剧...
3510,警惕投机拉高地价 虚假地王背后楼市红灯闪烁记者从湖南省建设厅了解到，1至6月，湖南省商品房竣...,警惕投机拉高地价虚假地王背后楼市红灯闪烁记者从湖南省建设厅了解到1至6月湖南省商品房竣工面积...


In [7]:
import jieba as jb
### 加载停用词
stopwords = get_stopwords_list("../data/baidu_stopwords.txt")
# 文本获取停用词
test_df['cuted_content']=test_df['cleaned_content'].apply(lambda x: " ".join([w for w in list(jb.cut(x)) if w not in stopwords]))
test_df.sample(10)

Building prefix dict from the default dictionary ...
Loading model from cache /var/folders/3n/5ywx4mgd3pz_9d58sjqmrxdm0000gn/T/jieba.cache
Loading model cost 0.710 seconds.
Prefix dict has been built successfully.


,content,cleaned_content,cuted_content
12781,组图：驻阿富汗英军“防弹内裤” 武装到命根子导语：信息时报综合报道 防弹内裤的点子或许引人发...,组图驻阿富汗英军防弹内裤武装到命根子导语信息时报综合报道防弹内裤的点子或许引人发噱但背后的意...,组图 驻 阿富汗 英军 防弹 内裤 武装 命根子 导语 信息时报 综合 报道 防弹 内裤 点...
10934,《文明online》冲锋夺奖结果公布史诗式即时战斗手机网游《文明online》复活转生冲级活...,文明online冲锋夺奖结果公布史诗式即时战斗手机网游文明online复活转生冲级活动已经降...,文明 online 冲锋 夺奖 公布 史诗式 即时 手机 网游 文明 online 复活 转...
19079,《掌魔》好友PK 赢话费大奖一个人玩游戏太没劲？来掌魔“好友挑战”发起挑战赛，邀请朋友们一块...,掌魔好友PK赢话费大奖一个人玩游戏太没劲来掌魔好友挑战发起挑战赛邀请朋友们一块来玩吧选择好友...,掌魔 好友 PK 赢 话费 大奖 一个 人 玩游戏 太 没劲 掌 魔 好友 挑战 发起 挑战...
655,12X光变防抖长焦 富士S1500带票仅1440 作者：陈亮 相机背面是一个2....,12X光变防抖长焦富士S1500带票仅1440作者陈亮相机背面是一个27英寸的广视角液晶取景...,12 X光 变防抖 长焦 富士 S1500 带票 仅 1440 作者 陈 亮相 机 背面 一...
6291,陈水扁父子本周将在法庭上对质中新网7月6日电 据台湾《中国时报》报道，台北地方法院审理陈水扁...,陈水扁父子本周将在法庭上对质中新网7月6日电据台湾中国时报报道台北地方法院审理陈水扁家庭弊案...,陈水扁 父子 本周 法庭 上 对质 中新网 7 月 6 日电 台湾 中国时报 报道 台北 地...
16669,约会《天劫ol》BOSS 勇敢说出你的爱今天，《天劫ol》甜蜜七夕缤纷活动“爱就说出来”正式...,约会天劫olBOSS勇敢说出你的爱今天天劫ol甜蜜七夕缤纷活动爱就说出来正式开启从即日到9月...,约会 天劫 olBOSS 勇敢 说出 爱 天劫 ol 甜蜜 七夕 缤纷 活动 爱 说 正式 ...
16420,2200点基金公司先跑 批量分红预防基民大撤退2008年以来成立的次新基金有122只，其中高...,2200点基金公司先跑批量分红预防基民大撤退2008年以来成立的次新基金有122只其中高达7...,2200 点 基金 公司 先跑 批量 分红 预防 基民 大 撤退 2008 年 成立 次 新...
12899,木家具新标准今起实施 具体定义各类木制家具今年“五一”，国家质量监督总局和国家标准化管理委员...,木家具新标准今起实施具体定义各类木制家具今年五一国家质量监督总局和国家标准化管理委员会共同发...,木家具 新 标准 今起 实施 定义 各类 木制家具 五一 国家 质量 监督 总局 国家 标准...
7929,《神兆OL》之PK不败的神话富有东方传奇色彩的大型多人在线横版手机网游巨作《神兆OL》已全面...,神兆OL之PK不败的神话富有东方传奇色彩的大型多人在线横版手机网游巨作神兆OL已全面推出最为...,神兆 OL PK 不败 神话 富有 东方 传奇色彩 大型 多人 在线 横版 手机 网游 巨作...
12858,美专家：易迎来上位机会 非实力因素令他跌出轮换新浪体育讯北京时间3月12日消息，据《华盛顿邮...,美专家易迎来上位机会非实力因素令他跌出轮换新浪体育讯北京时间3月12日消息据华盛顿邮报记者迈...,美 专家 易 迎来 上位 机会 非 实力 因素 令 跌 出 轮换 新浪 体育讯 北京 时间 ...


In [8]:
# 向量化文本
MAX_NUM_WORDS = 100000 # 最多保留不同词语词数，频率从大到小
MAX_SEQUENCE_LEN = 256
EMBEDDING_DIM=128

def tokenize(lang,max_num_words=None,max_sequence_len=300):
    lang_tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=max_num_words, 
    filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~', lower=True) # 
    lang_tokenizer.fit_on_texts(lang)

    tensor = lang_tokenizer.texts_to_sequences(lang)

    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor,maxlen=max_sequence_len,
                                                            padding='post',truncating='post') # NOTE
    print('* Total different words: %s.' % len(lang_tokenizer.word_index))
    return tensor, lang_tokenizer
# get X
X,_=tokenize(test_df['cuted_content'].values,MAX_NUM_WORDS,MAX_SEQUENCE_LEN)
print(X.shape)

* Total different words: 214108.
(20000, 256)


In [9]:
y_pred=model.predict(X)
y_pred.shape

(20000, 7)

In [ ]:
y_pred1 = y_pred.argmax(axis = 1)
y_pred1

In [35]:
#{0: '家居', 1: '房产', 2: '教育', 3: '时尚', 4: '时政', 5: '科技', 6: '财经',7: '游戏',8: '娱乐',9: '体育'}
#7类，分别为：财经、房产、家居、教育、科技、时尚、时政
#10个类别:财经、房产、家居、教育、科技、时尚、时政、游戏、娱乐、体育。
#{0: '可公开', 1: '低风险', 2: '中风险', 3: '高风险'}
#rank_label对应关系:0{0,8,9} 1{2,3,7} 2{1,5} 3{4,6}
def reflect_rank(line):
    res=-1
    if(line==0 or line==8 or line==9):
        res=0
    elif(line==2 or line==3 or line==7):
        res=1
    elif(line==1 or line==5):
        res=2
    elif(line==4 or line==6):
        res=3
    return res

cn_class_dic={0: '家居', 1: '房产', 2: '教育', 3: '时尚', 4: '时政', 5: '科技', 6: '财经',7: '游戏',8: '娱乐',9: '体育'}
cn_rank_dic={0: '可公开', 1: '低风险', 2: '中风险', 3: '高风险'}

def cn_class_label(line):
    return cn_class_dic[line]

def cn_rank_label(line):
    return cn_rank_dic[line]

In [36]:
#id class_label rank_label

test_pr=pd.DataFrame(y_pred1)
test_pr.columns = ['class_label']
test_pr['rank_label']=test_pr['class_label'].apply(reflect_rank)
test_pr['class_label']=test_pr['class_label'].apply(cn_class_label)
test_pr['rank_label']=test_pr['rank_label'].apply(cn_rank_label)

In [38]:
test_pr.to_csv('submission.csv',index_label='id')